In [51]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [44]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [45]:
#get all files path
posFiles = glob('review_polarity/txt_sentoken/pos/*')
negFiles = glob('review_polarity/txt_sentoken/neg/*')
#read text files
posReviews = np.array([open(f).read() for f in posFiles])
negReviews = np.array([open(f).read() for f in negFiles])
#use pandas to label and mix the data
polarity_files_df = pd.DataFrame({'pos':posReviews,'neg':negReviews})
polarity_files_df = pd.melt(polarity_files_df, value_vars=['pos','neg'],value_name="text",var_name="label")
polarity_files_df["label_num"] = polarity_files_df.label.map({"neg":0, "pos":1})
polarity_files_df.sample(5)

,label,text,label_num
1390,neg,if there were a subject just screaming to be m...,0
712,pos,""" rebel without a cause "" is such an importan...",1
1026,neg,"wolfgang petersen's latest , the perfect storm...",0
542,pos,"the keen wisdom of an elderly bank robber , th...",1
401,pos,the happy bastard's 30-second review : \nameri...,1


In [46]:
# split X and y into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(polarity_files_df.text, polarity_files_df.label_num, test_size=0.33, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1340,)
(660,)
(1340,)
(660,)


In [69]:
# remove English stop words
# include 1-grams and 2-grams (if 3-grams : seems not better because of the length of the vector)
# ignore terms that appear in more than 70% of the documents (intuitively meaningful, it is indeed the best multiple of 10% to have a good score )
# only keep terms that appear in at least 2 documents

# TODO : remove 'not' etc. from stopwords to not remove 'not' from sentences like 'this film is not bad'
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english', ngram_range =(1,2), max_df =0.7, min_df=2)),
                     ('nb', MultinomialNB()),
                    ])

text_clf = text_clf.fit(X_train, y_train)
y_pred = text_clf.predict(X_test)

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
# à vérifier : pos et neg sont biens les pos et neg
print(metrics.classification_report(y_test, y_pred,
    target_names=["pos","neg"]))
metrics.accuracy_score(y_test, y_pred)

# Plot non-normalized confusion matrix
#plt.figure()
#plot_confusion_matrix(cnf_matrix, classes=['pos','neg'],
#                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
#plt.figure()
#plot_confusion_matrix(cnf_matrix, classes=['pos','neg'], normalize=True,
#                      title='Normalized confusion matrix')

#plt.show()

              precision    recall  f1-score   support

         pos       0.82      0.80      0.81       335
         neg       0.80      0.82      0.81       325

   micro avg       0.81      0.81      0.81       660
   macro avg       0.81      0.81      0.81       660
weighted avg       0.81      0.81      0.81       660



0.80909090909090908

In [52]:
#test a range of hyperparameters
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english', ngram_range =(1,3), max_df =0.7, min_df=2)),
                     ('tfidf', TfidfTransformer()),
                     ('nb', MultinomialNB()),
                    ])

text_clf.get_params()

{'memory': None,
 'nb': MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
 'nb__alpha': 1.0,
 'nb__class_prior': None,
 'nb__fit_prior': True,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=0.7, max_features=None, min_df=2,
           ngram_range=(1, 3), preprocessor=None, stop_words='english',
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
 'tfidf': TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True),
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': True,
 'vect': CountVectorizer(analyzer='word', binary=False, decode_e

In [ ]:
from sklearn.model_selection import GridSearchCV
#TO ADAPT depending on what text_clf.get_params() displays
#parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
#              'tfidf__use_idf': (True, False),
#              'nb__alpha': (1e-2, 1e-3),
#}
#gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1, return_train_score=True)

In [ ]:
gs_clf = gs_clf.fit(X_train, y_train)
y_pred = gs_clf.predict(X_test)

gs_clf.best_score_  

In [ ]:
##TO ADAPT : examine most frequent words assigned to pos and neg reviews

# store the vocabulary of X_train
X_train_tokens = vect.get_feature_names()
len(X_train_tokens)
# examine the first 50 tokens
print(X_train_tokens[0:50])
# examine the last 50 tokens
print(X_train_tokens[-50:])
# Naive Bayes counts the number of times each token appears in each class
# trailing underscore is scikit convention for attributes that are learned during model fitting
nb.feature_count_
# rows represent classes, columns represent tokens
nb.feature_count_.shape
# number of times each token appears across all HAM messages
ham_token_count = nb.feature_count_[0, :]
ham_token_count
# number of times each token appears across all SPAM messages
spam_token_count = nb.feature_count_[1, :]
spam_token_count
# create a DataFrame of tokens with their separate ham and spam counts
tokens = pd.DataFrame({"token":X_train_tokens, "ham":ham_token_count, "spam":spam_token_count}).set_index("token")
tokens.head()
# examine 5 random DataFrame rows
tokens.sample(5, random_state=6)
# Naive Bayes counts the number of observations in each class
nb.class_count_
# add 1 to ham and spam counts to avoid 0 probabilities
tokens['ham'] = tokens['ham'] + 1
tokens['spam'] = tokens['spam'] + 1
tokens.sample(5, random_state=6)
# convert the ham and spam counts into frequencies
tokens['ham'] = tokens['ham'] / nb.class_count_[0]
tokens['spam'] = tokens['spam'] / nb.class_count_[1]
tokens.sample(5, random_state=6)
# calculate the ratio of spam-to-ham for each token
tokens['spam_ratio'] = tokens['spam'] / tokens['ham']
tokens.sample(5, random_state=6)
# examine the DataFrame sorted by spam_ratio
# note: use sort() instead of sort_values() for pandas 0.16.2 and earlier
tokens.sort_values('spam_ratio', ascending=False)
# look up the spam_ratio for a given token
tokens.loc["dating", "spam_ratio"]

In [ ]:
# Logistic regression
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english', ngram_range =(1,3), max_df =0.7, min_df=2)),
                     ('nb', LogisticRegression()),
                    ])

y_pred_class = text_clf.predict(X_test)

y_pred_prob = text_clf.predict(X_test)

metrics.confusion_matrix(y_test, y_pred_class)

print(metrics.classification_report(y_test, y_pred_class,
    target_names=["pos","neg"]))
metrics.accuracy_score(y_test, y_pred_class)
metrics.roc_auc_score(y_test, y_pred_prob)

In [ ]:
# With TF
from sklearn.feature_extraction.text import TfidfTransformer

text_clf = Pipeline([('tfidf', TfidfTransformer(use_idf=False)),
                     ('nb', LogisticRegression()),
                    ])

X_train_tf = text_clf.fit_transform(X_train)
X_train_tf.shape


tf_transformer = TfidfTransformer(use_idf=False)
X_test_tf = tfidf_transformer.transform(X_test)

y_pred_class = text_clf.predict(X_test_tf)

y_pred_prob = text_clf.predict(X_test_tf)

metrics.confusion_matrix(y_test, y_pred_class)

print(metrics.classification_report(y_test, y_pred_class,
    target_names=["pos","neg"]))
metrics.accuracy_score(y_test, y_pred_class)
metrics.roc_auc_score(y_test, y_pred_prob)

In [ ]:
# Computing TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

text_clf = Pipeline([('tfidf', TfidfTransformer()),
                     ('nb', LogisticRegression()),
                    ])

X_train_tfidf = text_clf.fit_transform(X_train)
X_train_tfidf.shape


tf_transformer = TfidfTransformer()
X_test_tfidf = tfidf_transformer.transform(X_test)

y_pred_class = text_clf.predict(X_test_tfidf)

y_pred_prob = text_clf.predict(X_test_tfidf)

metrics.confusion_matrix(y_test, y_pred_class)

print(metrics.classification_report(y_test, y_pred_class,
    target_names=["pos","neg"]))
metrics.accuracy_score(y_test, y_pred_class)
metrics.roc_auc_score(y_test, y_pred_prob)

In [ ]:
#USE STRING KERNELS : not done until now in others group and suggested at the very end of the teacher's notebook